In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import string
import csv
%matplotlib inline

In [ ]:
import sys
import six
import json
import requests

named_entities = ["storageLink","entity", "type", "count", "entity_start_offset", "entity_end_offset", "relevance", "subtypes", "DBpedia_URL", "sentiment_label", "sentiment_score", "emotion_sadness", "emotion_fear", "emotion_joy", "emotion_disgust", "emotion_anger"]

url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2018-11-16'
key = [PLACEHOLDER_API_KEY]

videos = [PLACEHOLDER_LIST_OF_VIDEO_IDS]

headers = {'Content-Type': 'application/json'}
parameters = {}
parameters['features'] = {}
parameters['features']['entities'] = {}
parameters['features']['entities']['mentions'] = True
parameters['features']['entities']['sentiment'] = True
parameters['features']['entities']['emotion'] = True
parameters['language'] = 'en'

for video in videos:
    
    with open("..data/video_subtitles_entities/" + video + ".csv", "w" ) as outSentences:
        writer = csv.writer( outSentences )
        writer.writerow(named_entities)

    video_subtitle_file = [PLACEHOLDER_VIDEO_SUBTITLE_CSV]

    df = video_subtitle_file.groupby(by=['storageLink','transcript','transcript_part'], as_index=False).first()
    df = df[["storageLink", "transcript", "transcript_part", "start_time", "end_time"]]
    df = df.sort_values(["transcript_part"])

    text = " ".join(list(df["transcript"]))

    parameters['text'] = text

    data = json.dumps(parameters)
    request = requests.post(url, headers=headers, data=data, auth=('apikey', ))        
    json_object = request.json()
    
    
    if 'entities' in json_object:  
        print(json_object['entities'])
        for entity in json_object['entities']:
            if "mentions" in entity:
                for mention in entity['mentions']:
                    processedList = [video]
                    processedList.append(entity['text'].encode('utf-8').decode('utf-8'))
                    processedList.append(entity['type'])
                    processedList.append(entity['count'])
                    processedList.append(mention['location'][0])
                    processedList.append(mention['location'][1])
                    processedList.append(entity['relevance'])
                    if 'disambiguation' in entity:
                        if 'subtype' in entity['disambiguation']:
                            processedList.append(", ".join(entity['disambiguation']['subtype']))
                        else:
                            processedList.append("NONE")
                        if 'dbpedia_resource' in entity['disambiguation']:
                            processedList.append(entity['disambiguation']['dbpedia_resource'])
                        else:
                            processedList.append("NONE")
                    else:
                        processedList.append("NONE")
                        processedList.append("NONE")
                    processedList.append(entity['sentiment']['label'])
                    processedList.append(entity['sentiment']['score'])

                    if 'emotion' in entity:
                        processedList.append(entity['emotion']['sadness'])
                        processedList.append(entity['emotion']['fear'])
                        processedList.append(entity['emotion']['joy'])
                        processedList.append(entity['emotion']['disgust'])
                        processedList.append(entity['emotion']['anger'])
                    else:
                        processedList.append(0.0)
                        processedList.append(0.0)
                        processedList.append(0.0)
                        processedList.append(0.0)
                        processedList.append(0.0)

                    with open("..data/video_subtitles_entities/" + video + ".csv",, "a") as outSentences:
                        writer = csv.writer( outSentences )
                        writer.writerow(processedList)
